## Packages and params

In [ ]:
#Packages
from ipynb_pkgs import *
import utils as clf_utils
from utils import params as clf_params
from forecasting.analysis import *

# Data preprocess

In [ ]:
data, features, features_predict, center = load.load_data(species='Southern elephant seal')

## Store data for aniMotum (R)

Run this cell before training the animotum models (scripts/animotum_models.R)

In [ ]:
data_train, data_val, data_test = load.preprocess_animotum()

# Data visualization and split

In [ ]:
from forecasting.plots import dataset
# paper_fig
dataset.dataset_split(ext='pdf')

In [ ]:
# paper fig
dataset.feature_multicollinearity(corr_method='pearson', ext=['pdf', 'png'])

# Hyperparameter tuning

Run first scripts/forecast_optim.py or scripts/imputation_optim.py

In [ ]:
optq = OptimalHyperparameters(mode='quantile')
optq.best()

In [ ]:
optq.optim_params

In [ ]:
# best HP forecasting
get_hp(store_missing_idxs=True, max_train_days=4, params_idx=0)

In [ ]:
# best HP imputation
get_hp(predict_shift=112, max_train_days=4, store_missing_idxs=True, expand_encoder_until_future_length=True, task='imputation', epochs=200, params_idx=0)

In [ ]:
# best HP TFT[B]
get_hp(mode='quantile', task='forecasting', **params.TFT_specs['forecasting'], max_train_days=4, quantiles='all', s_q=5, params_idx=0)

In [ ]:
get_hp(mode='quantile', task='imputation', **params.TFT_specs['imputation'], quantiles='all', s_q=1, params_idx=1)

# TFT

## Example

In [ ]:
# tft = model.QuantileForecaster(store_missing_idxs=True, max_train_days=4)
tft = model.QuantileForecaster(store_missing_idxs=True, max_train_days=4, quantiles='all', s_q=1) # TFT 2D distribution (TFT[B])

In [ ]:
tft.train(epochs=2, limit_train_batches=2)

In [ ]:
results = tft.get_results(partition='test')

In [ ]:
results.keys()

## Evals

Requires having trained before (scripts/forecast_store.py). If using hyperparameter optimization, it requires first computing the HP (scripts/forecast_optim.py)

In [ ]:
cvg, area = area_coverage_CI(params_idx=0)

In [ ]:
# quality
kwargs = dict(mpl_val=True, partition='test')
df = pd.concat([quality_sample(params_idx=p, **kwargs).mean() for p in range(6)], axis=1).T

In [ ]:
# best model
task = 'forecasting'
best_idx, _ = quantile_best_model(task=task, **params.TFT_specs[task])

# SSMs

In [ ]:
for model in tqdm(params.ssm_models):
    for magnitude in ['area', 'coverage']:
        for se_val_fit in ['best']: #[True, False]:
            df = ssm.eval_ssm_CI(model=model, magnitude=magnitude, se_val_fit=se_val_fit)

# Main

## SSMs vs TFT

In [ ]:
# paper tables
# task in ['forecasting', 'imputation']
# Disable PR optimization with mpl_val=False
df = main_results(task='forecasting', mpl_val=True)
criteria = {col: ('upper' if 'Q' in col else 'lower') for col in df.columns}
df = pd_utils.highlight_best(df, criteria=criteria)

## Prediction visualization

In [ ]:
# paper fig
for method in ['hull', 'contour']:
    for animal in [37, 199]:
        plot_pr_comparison_trajectory(animal=animal, task='forecasting', method=method,
                                      ext=['pdf', 'svg'])

## SHAP values

In [ ]:
# paper fig
ext = ['svg', 'pdf']
error_analysis.shap_plot(target='distance', task='forecasting', all_features=False, c=True, exclude_prediction_attrs=True, xlims=[-20,20], ext=ext) # 9% decrease in error, 18% in val

error_analysis.shap_plot(target='area', task='forecasting', all_features=False, c=False, exclude_prediction_attrs=True, xlims=[int(-2.5e5), int(2.5e5)], ext=ext) # 19% decrease in error, 72% in val

In [ ]:
# paper fig
ext = ['svg', 'pdf']
error_analysis.shap_plot(target='distance', task='imputation', all_features=False, exclude_prediction_attrs=True, c=True, xlims=[-11, 11], ext=ext)
error_analysis.shap_plot(target='area', task='imputation', all_features=False, exclude_prediction_attrs=True, c=True, xlims=[-20002, 20002], ext=ext)

# Supplementary

## SSMs vs TFT

In [ ]:
ext = ['png', 'svg']
for metric in ['CI']:#, 'rae']:#, 'rae', 'arae']:
    for task in ['forecasting', 'imputation']:
        model_comp_coverage(metric=metric, ext=ext, task=task, **params.TFT_specs[task], add_bivariate=True)

In [ ]:
ext = ['png', 'svg']
for task in ['forecasting', 'imputation']:
    model_comp_area(ext=ext, task=task, **params.TFT_specs[task], add_bivariate=True)

In [ ]:
ylim = [0, 550]
ext = ['png', 'svg']
for task in ['forecasting']:#, 'imputation']:
    point_prediction_error_plot(ext=ext, task=task, ylim=ylim, **params.TFT_specs[task], add_bivariate=True)

In [ ]:
# table supplementary
df = point_prediction_best_models_across_time()

In [ ]:
df

In [ ]:
# supplementary tables (area, coverage, distance)
df = aggregate_summary(unit=False, divide_area=1e5, simplify=False, area_exp_fmt=False,
                                task='forecasting', CI_expansion=True)
df = pd_utils.highlight_best(df, 'lower')

## Prediction visualization

In [ ]:
ext = ['svg', 'png']
specs = dict(lw=5, ms=34, ms_p=42, lw_p=5, mlw=3)
trajectory_confidence_region(step=3, legend=True, ext=ext, **specs) # legend
trajectory_confidence_region(step=-1, legend=False, ext=ext) #initial_trajectory

In [ ]:
specs = dict(lw=5, ms=34, ms_p=42, lw_p=5, mlw=3)
ext = ['svg', 'png']
trajectory_confidence_region(step=0, legend=False, ext=ext, n_obs=7, n_obs_lims=5, **specs) # t1
trajectory_confidence_region(step=1, legend=False, ext=ext, n_obs=7, n_obs_lims=5, **specs) # t2
trajectory_confidence_region(step=5, legend=False, ext=ext, n_obs=7, n_obs_lims=5,
                                      title_type='step_n', **specs) # t_n

In [ ]:
# texts
trajectory_confidence_region(step=0, legend=False, ext='svg', n_obs=7, n_obs_lims=5, text=True) # t1
trajectory_confidence_region(step=1, legend=False, ext='svg', n_obs=7, n_obs_lims=5, text=True) # t2
trajectory_confidence_region(step=5, legend=False, ext='svg', n_obs=7, n_obs_lims=5,
                                      title_type='step_n', text=True) # t_n

## Variable selection weights and attention

In [ ]:
# paper fig
params_idx = 'best'
ext = ['svg', 'png']
y_max = 0.30 # None
for task in ['forecasting', 'imputation']:
    kwargs = params.TFT_specs[task]
    attention_plot(params_idx=params_idx, ext=ext, task=task, y_max=y_max, **kwargs)

In [ ]:
# paper fig
ext = ['png', 'svg']
params_idx = 'best'
var_offset = {'encoder': (0.2, [-0.3, 0.51]),
              'decoder': (0.2, [-0.5, 1.05]),
              'static': (0.45, [-0.5, 1.05])}
for task in ['imputation']: #['forecasting', 'imputation']:
    kwargs = params.TFT_specs[task]
    if task == 'imputation':
        var_offset['future'] = (0.2, [-0.3, 0.51])
        var_offset['static'] = (0.45, [-0.8, 1.05])
    for var_type, (offset, xlims) in var_offset.items():
        xlims[1] = 1.05
        if task != 'imputation' or var_type != 'static':
            xlims[0] = -0.5
        if var_type in ['encoder', 'future']:
            feature_importance_plot(task=task, params_idx=params_idx, var_type=var_type, ext=ext, offset=offset, xlims=xlims, **kwargs)
        else:
            feature_importance_plot(task=task, params_idx=params_idx, var_type=var_type, ext=ext, offset=offset, xlims=xlims, **kwargs)

## Error analysis

In [ ]:
# paper supplementary
for method in ['spearman']: #['spearman', 'pearson']:
    for target in ['Q', 'distance']:
        error_analysis.error_corr_plot(task='forecasting', method=method, target=target, ext=['png', 'svg'], min_corr=0.3)

In [ ]:
# paper result
df = error_analysis.error_corr_with_target(target='distance', task='forecasting', **params.TFT_specs['forecasting'])
# distance-area correlation: 0.57
df.loc['area', 'corr']

In [ ]:
# paper result
task = 'imputation'
df = error_analysis.error_corr_with_target(target='distance', ef_abs_diff='replace', task=task, **params.TFT_specs[task])
# distance-area correlation: 0.77
df.loc['area', 'corr']

In [ ]:
# paper supplementary
error_analysis.bathymetry_speed_against_distance(ext='png')

In [ ]:
# paper supplementary
error_analysis.bathymetry_speed_pmf_corr_with_distance()

In [ ]:
error_analysis.distance_error_heatmap(task='forecasting', ext=['svg', 'png'])

In [ ]:
error_analysis.distance_error_contour(task='forecasting', ext=['svg', 'png'])

In [ ]:
task = 'forecasting'
error_avg, bathymetry_rescaled, speed, cds_encoder, cds_decoder = error_analysis.preprocess_distance_analysis(task=task)
speed.median(), speed.quantile(0.8)

In [ ]:
np.median(bathymetry_rescaled), np.percentile(bathymetry_rescaled, [25, 60])

In [ ]:
# paper supplementary: Compare errors vs bathymetry
best_zone = (bathymetry_rescaled.squeeze() >= -1000) & (speed <= 3)
error_avg[best_zone].mean(), error_avg[~best_zone].mean()

In [ ]:
bootstrap.CI_bca(error_avg[best_zone].values, custom_metrics.nb_mean, R=int(1e4)), bootstrap.CI_bca(error_avg[~best_zone].values, custom_metrics.nb_mean, R=int(1e4))

In [ ]:
bad_zone_low_speed = (bathymetry_rescaled.squeeze() < -1000) & (speed <= 3)
error_avg[bad_zone_low_speed].mean(), bootstrap.CI_bca(error_avg[bad_zone_low_speed].values, custom_metrics.nb_mean, R=int(1e4))